###  Ybigta 19기 신입기수프로젝트 5조 와인추천시스템 중 Tokenizing부분입니다

In [ ]:
#pip install konlpy

In [2]:
import pandas as pd
import numpy as np
import konlpy
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt,Kkma
okt = Okt()
kkma = Kkma()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
wine = pd.read_csv("/content/drive/My Drive/Colab Notebooks/team project/total.csv")  

In [3]:
wine = pd.read_csv("total_.csv")
wine.head()

,Unnamed: 0,name_ko,name_en,price,type,country,city,aroma,food,rate_expert,...,body,tannin,maker's_note,tasting_note,posting_title,posting_text,wine_title,wine_text,hashtag,blog_text
0,0,더 롱 리틀 독 스파클링,The Long Little Dog Sparkling,30000.0,스파클링,프랑스,랑그독 루씨용,"복숭아, 살구, 꽃, 흰꽃, 레몬","리코타, 파스타, 샐러드",-,...,2.0,1.0,"밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",NaN,NaN,NaN,NaN,NaN,NaN,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,1,루베롱 블랑,LUBERON BLANC,25000.0,NaN,프랑스,론,"헤이즐넛, 꽃, 아카시아, 레몬",NaN,-,...,3.0,1.0,"화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",NaN,NaN,NaN,NaN,NaN,NaN,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼","Marie Stuart, Cuvee de la Reine Brut",145000.0,스파클링,프랑스,샹파뉴,꽃,"아페리티프, 파스타",-,...,3.0,1.0,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",NaN,와인21닷컴이 선정한 7월 이달의 와인,조금 늦게 찾아온 장마와 무더위가 기다리고 있는 7월. 장마철의 습한 기운을 떨쳐내...,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,NaN,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,3,"마스카 델 타코, 로시 피노 네로 로사토","Masca del Tacco, “ Ro’Si ” Pinot Nero Rosato",38000.0,NaN,이탈리아,뿔리아,"딸기, 미네랄",NaN,-,...,2.0,1.0,밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,NaN,NaN,NaN,NaN,NaN,NaN,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,4,"비에티, 바르베라 다스티 '트레 비그네'","Vietti, Barbera d'Asti 'Tre Vigne'",50000.0,NaN,이탈리아,아스티,"체리, 오크",NaN,-,...,3.0,4.0,바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,NaN,NaN,NaN,NaN,NaN,NaN,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


In [4]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     806 non-null    int64  
 1   name_ko        805 non-null    object 
 2   name_en        805 non-null    object 
 3   price          304 non-null    float64
 4   type           68 non-null     object 
 5   country        325 non-null    object 
 6   city           309 non-null    object 
 7   aroma          321 non-null    object 
 8   food           214 non-null    object 
 9   rate_expert    325 non-null    object 
 10  rate_consumer  325 non-null    object 
 11  alcohol        241 non-null    object 
 12  sugar          325 non-null    float64
 13  acidity        325 non-null    float64
 14  body           325 non-null    float64
 15  tannin         325 non-null    float64
 16  maker's_note   317 non-null    object 
 17  tasting_note   28 non-null     object 
 18  posting_ti

###### 먼저, 수치형 데이터 보다는 블로그나 기사 등을 통해 특징을 추출하는 것이 목표이기 때문에 해당 열들을 지웠습니다.

###### posting_text의 경우, 불필요한 말이 너무 많아 오히려 분석의 성능을 저하시킨다고 판단해 지웠습니다

In [5]:
del wine["Unnamed: 0"]
del wine["name_en"]
del wine["price"]
del wine["country"]
del wine["city"]
del wine["rate_expert"]
del wine["rate_consumer"]
del wine["sugar"]
del wine["alcohol"]
del wine["acidity"]
del wine["tannin"]
del wine["posting_text"]
del wine["food"]
del wine["body"]

In [6]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",NaN,NaN,NaN,NaN,NaN,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,루베롱 블랑,NaN,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",NaN,NaN,NaN,NaN,NaN,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",NaN,와인21닷컴이 선정한 7월 이달의 와인,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,NaN,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,"마스카 델 타코, 로시 피노 네로 로사토",NaN,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,NaN,NaN,NaN,NaN,NaN,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,"비에티, 바르베라 다스티 '트레 비그네'",NaN,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,NaN,NaN,NaN,NaN,NaN,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


###### 분석의 성능을 위해 반복되는 불필요한 어구들을 정규표현식을 사용해 지웠습니다. (기자이름, 기사를 쓴 날, 평점 등)

In [7]:
import re

In [8]:
for i  in range(0, 806):
    wine.tasting_note[i] = re.sub("\n.{1,}\n[0-9]{4}[.][0-9]{2}[.][0-9]{2}\n[A-Z0-9]{2,}\s빈티지\n[0-9][.][0-9]"," ", str(wine.tasting_note[i]))

In [9]:
wine.posting_title.unique()

array([nan, '와인21닷컴이 선정한 7월 이달의 와인', '와인21추천 BEST OF BEST, 선물하기 좋은 와인',
       '올해 밸런타인데이에 함께할 와인!', '졸업 시즌의 축하주, 스파클링 와인',
       '와인21추천 BEST OF BEST, 꼭 한번 마셔볼만한 마트 와인', '초여름밤, 설레는 기분을 선사해줄 와인',
       "와인21추천 BEST OF BEST, 집에서 더 즐겁게 '홈파티 와인'", '늦여름에 마시기 좋은 레드 와인',
       '와인21추천 BEST OF BEST - 치명적 우아함, 부르고뉴(Bourgogne)',
       '[와인추천] 소소한 야외 파티에서 즐기기 좋은 와인', '와인21닷컴이 선정한 3월 이달의 와인',
       '부부의 날을 기념하며 마시기 좋은 와인', '와인21닷컴이 선정한 2월 이달의 와인',
       '호캉스에 단 한 병의 와인을 가지고 간다면?', '와인21닷컴이 선정한 5월 이달의 와인',
       '와인21닷컴이 선정한 6월 이달의 와인', '와인21닷컴이 선정한 8월 이달의 와인',
       '와인21닷컴이 선정한 9월 이달의 와인', '와인21닷컴이 선정한 10월 이달의 와인',
       '와인21닷컴이 선정한 11월 이달의 와인', '와인21닷컴이 선정한 12월 이달의 와인',
       '와인21닷컴이 선정한 2021년 1월 이달의 와인', '와인21닷컴이 선정한 4월 이달의 와인',
       '폭염이 지속되는 여름밤에 마시고 싶은 와인', '무더운 날 칠링해서 마시기 좋은 레드 와인은?',
       '삼계탕과 잘 어울리는 와인은?', '봄바람과 함께, 야외에서 즐기는 와인',
       '행사가 많은 5월, 선물이 고민인가요?', '춘곤증을 날려주는 청량감 넘치는 와인',
       '미세먼지가 심한 날, 홈술 추천 와인', '벚꽃 필 무렵엔 이 와인을!', '봄날, 지금 썸타는 중이라면?',
       "와인21's PI

In [10]:
for i  in range(0, 806):
    wine.posting_title[i] = re.sub("와인21추천"," ", str(wine.posting_title[i]))
    wine.posting_title[i] = re.sub("이달의 와인"," ", str(wine.posting_title[i]))
    wine.posting_title[i] = re.sub("BEST OF BEST"," ", str(wine.posting_title[i]))    
    wine.posting_title[i] = re.sub("와인"," ", str(wine.posting_title[i]))    
    wine.posting_title[i] = re.sub("21닷컴이 선정한"," ", str(wine.posting_title[i]))        

###### 불필요하다고 판단된 어구들이 지워진 이후 unique 값들을 확인해보았습니다

In [11]:
wine.tasting_note.unique()

array(['nan', ' \n세인트 넘 조아요~~~~\n\n굿뜨베리...', '  \n샤도네이를 좋아하게 해준 와인',
       ' \n양고기 허브솔트와 마리아주',
       ' \n오픈직후 피어오르는 잘익은 과실향이 인상적이나 한모금하는순간 할퀴는듯한 탄닌이 길들여지지 않은야생의 맹수같은 느낌이다. 몇년의 기간을 두고 마시면 깜짝놀랄만한 녀석으로 변할것같다고 조심스레 예상해본다.\n마시자마자 든 느낌은 아주 기름기가 많은 육류와 함께하면 좋을듯하다',
       ' \n진지하면서도 위트가 넘치는 흥미로운 와인. 오레곤을 제외한 미국와인의 특성상 75%이상 사용된 주품종은 레이블에 명시할 수 있는 와인법에 따라 분명 피노누아 이외의 다른 품종이 블렌딩 된 것 같은데 감을 잡을 수 없었다. 찾아보니 글로벌~한 블렌딩. 딸기, 섬세한 체다치즈, 바닐라, 캬라멜 등의 밝은 이미지의 아로마 뒤에 보여지는 우아한 빨레는 이러한 블렌딩의 마법인것 같기도 하다. 우연히 견과류와 함께했는데 크렌베리,캐슈넛과의 시너지 효과가 꽤나 좋았다. 호두는 껍질이 탄닌과 충돌해 에러:)',
       ' \n만들어 먹는 뱅쇼는 그때그때 맛도 다르고 잘못해서 끓기라도 하면 맛이 틀어지는데 크리스트 킨들스는 병째 데워 마시면 온기가 퍼지고 맛 역시 튀는곳 없이 무난해서 좋았다.\n마무리로 올라오는 상큼한 과일맛이 일품 \n전반적으로 산뜻하고 깔끔한 맛~\n너무 강하지도 약하지도 않은 바디감과 산미....매우 괜찮았음\n그냥 데워만 마셔도 좋겠지만 레몬 조각을 살짝 띄워서 마시니 더욱 맛있었는듯..^^',
       ' \n1804 tasting at coex expo \n1804 tasting at coex expo',
       ' \n차갑게 칠링한 로제 스파클링 와인은 특유의 청량감으로 몸과 마음에 신선한 기운을 불어넣어 준다. No.1 호주 와인 브랜드인 제이콥스 크릭이 만든 로제 스파클링은 제철 과일인 딸기나 참외 등과 잘 어울린다. 또, 캠핑에서 즐길 수 있는 조

In [12]:
wine.posting_title.unique()

array(['nan', '   7월  ', '   , 선물하기 좋은  ', '올해 밸런타인데이에 함께할  !',
       '졸업 시즌의 축하주, 스파클링  ', '   , 꼭 한번 마셔볼만한 마트  ',
       '초여름밤, 설레는 기분을 선사해줄  ', "   , 집에서 더 즐겁게 '홈파티  '",
       '늦여름에 마시기 좋은 레드  ', '    - 치명적 우아함, 부르고뉴(Bourgogne)',
       '[ 추천] 소소한 야외 파티에서 즐기기 좋은  ', '   3월  ', '부부의 날을 기념하며 마시기 좋은  ',
       '   2월  ', '호캉스에 단 한 병의  을 가지고 간다면?', '   5월  ', '   6월  ',
       '   8월  ', '   9월  ', '   10월  ', '   11월  ', '   12월  ',
       '   2021년 1월  ', '   4월  ', '폭염이 지속되는 여름밤에 마시고 싶은  ',
       '무더운 날 칠링해서 마시기 좋은 레드  은?', '삼계탕과 잘 어울리는  은?',
       '봄바람과 함께, 야외에서 즐기는  ', '행사가 많은 5월, 선물이 고민인가요?',
       '춘곤증을 날려주는 청량감 넘치는  ', '미세먼지가 심한 날, 홈술 추천  ', '벚꽃 필 무렵엔 이  을!',
       '봄날, 지금 썸타는 중이라면?', " 21's PICK, 카베르네 소비뇽(Cabernet Sauvignon)편",
       '봄을 기다리며 즐기기 좋은  ', '설날 선물을 위한 추천  ', '   , 2021년 ‘소중한  들',
       '이제는 버블을 즐길 시간!', '홈스토랑에서 오픈할  은?',
       '   , 프리미티보 & 진판델(Primitivo & Zinfandel)', '기분전환을 위한 추천  ',
       '찬바람이 불 때는 이  을!', '할로윈을 더 재미있게 만들어줄  ', '차박은 이  과 함께!',
       

In [13]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",nan,nan,NaN,NaN,NaN,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,루베롱 블랑,NaN,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",nan,nan,NaN,NaN,NaN,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",nan,7월,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,NaN,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,"마스카 델 타코, 로시 피노 네로 로사토",NaN,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,nan,nan,NaN,NaN,NaN,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,"비에티, 바르베라 다스티 '트레 비그네'",NaN,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,nan,nan,NaN,NaN,NaN,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


In [14]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name_ko        805 non-null    object
 1   type           68 non-null     object
 2   aroma          321 non-null    object
 3   maker's_note   317 non-null    object
 4   tasting_note   806 non-null    object
 5   posting_title  806 non-null    object
 6   wine_title     290 non-null    object
 7   wine_text      582 non-null    object
 8   hashtag        25 non-null     object
 9   blog_text      301 non-null    object
dtypes: object(10)
memory usage: 63.1+ KB


###### blog_text와 blog_text들을 제외한 모든 열을 합친 새로운 열로 만드는 것이 목표!
###### 이 때, null값이 있으면 합치기 어렵기 때문에 null값에 tokenize 할 때 쉽게 제거할 수 있는 의미 없는 값을 넣음

In [15]:
wine = wine.fillna("ok")

In [16]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",nan,nan,ok,ok,ok,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,루베롱 블랑,ok,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",nan,nan,ok,ok,ok,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",nan,7월,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,ok,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,"마스카 델 타코, 로시 피노 네로 로사토",ok,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,nan,nan,ok,ok,ok,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,"비에티, 바르베라 다스티 '트레 비그네'",ok,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,nan,nan,ok,ok,ok,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


In [17]:
wine["tasting_note"].isnull().sum()

0

###### 이유는 모르겠으나, fillna로 채워지지 않는 np.nan값은 replace 함수를 써서 대체

In [18]:
wine.replace("nan", "ok", inplace=True)

###### 아래 열들을 합쳐 하나의 열로 만듦

In [19]:
wine["review"] = wine["type"] +"." + wine["aroma"] +"."+ wine["maker's_note"] +"."+ wine["tasting_note"] +"."+ wine["posting_title"] +"."+ wine["wine_title"] +"."+ wine["wine_text"] +"."+ wine["hashtag"] +"."

In [20]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text,review
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",ok,ok,ok,ok,ok,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복..."
1,루베롱 블랑,ok,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",ok,ok,ok,ok,ok,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...,"ok.헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",ok,7월,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,ok,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달..."
3,"마스카 델 타코, 로시 피노 네로 로사토",ok,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,ok,ok,ok,ok,ok,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...,"ok.딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍..."
4,"비에티, 바르베라 다스티 '트레 비그네'",ok,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,ok,ok,ok,ok,ok,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne...","ok.체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕..."


###### 필요없어진 열 삭제

In [21]:
del wine["type"]
del wine["aroma"]
del wine["maker's_note"]
del wine["tasting_note"]
del wine["wine_title"]
del wine["wine_text"]
del wine["hashtag"]
del wine["posting_title"]

In [22]:
wine.head()

,name_ko,blog_text,review
0,더 롱 리틀 독 스파클링,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복..."
1,루베롱 블랑,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...,"ok.헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...,"ok.딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍..."
4,"비에티, 바르베라 다스티 '트레 비그네'","(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne...","ok.체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕..."


###### 영어와 기호 의미없는 줄바꿈이 너무 많아 토크나이즈가 제대로 되지 않아, 한글과 숫자 약간의 기호를 제외한 글자들은 모두 삭제함

In [23]:
wine["blog_text"] = wine["blog_text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]","")

<ipython-input-23-d412dda28f9d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  wine["blog_text"] = wine["blog_text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]","")


In [24]:
wine["review"] = wine["review"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9.,]","")

<ipython-input-24-63d9c72f23f1>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  wine["review"] = wine["review"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9.,]","")


In [25]:
wine.head()

,name_ko,blog_text,review
0,더 롱 리틀 독 스파클링,더롱리틀독스파클링소유리20216291549복사이웃추가더롱리틀독스파클링단맛이없고깔끔한...,"스파클링.복숭아,살구,꽃,흰꽃,레몬.밝고맑은레몬컬러에레몬,살구,복숭아,흰꽃의아로마가..."
1,루베롱 블랑,와인이야기하남포도내음와인아울렛와인샵와인소개화이트와인추천마레농클래식루베롱블랑2016포...,".헤이즐넛,꽃,아카시아,레몬.화사한노란색을띠며레몬,아카시아꽃,헤이즐넛향이어우러진다...."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천와인21닷컴이선정한7월이달의와인와인21202171950복사이웃추가조금늦게찾아...,"스파클링.꽃.반짝이는연한금빛의정교한거품을지녔으며,흰꽃패션프루츠,달콤한흰과일과은은한미..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인마스카델타코제이픽와인산타20216212026복사이웃추가마스카델타코는이탈리...,".딸기,미네랄.밟은옅은분홍색에야생딸기젤리수박과같은달콤한향이풍부하고그뒤로향신료광물미네..."
4,"비에티, 바르베라 다스티 '트레 비그네'",2018비에띠바르베라다스티트레비녜2018데미안202142700복사이웃추가2018비에...,".체리,오크.바르베라다스티의포도는라크레나포도밭으로유명한아스티언덕주변인아그리아노테르메..."


###### review 열을  토큰화한 뒤 원하는 품사를 가진 단어만 남겨둘 새로운 열 추가

In [26]:
wine["review_token"] = " "

###### 원하는 품사(의미를 가진 품사)만 새로운 열에 추가함

In [27]:
for i in range(0,806):
    pos = okt.pos(wine.review[i])
    for j in pos:
        if j[1] == "Adjective":
            wine.review_token[i] = wine.review_token[i] + " " + j[0] 
        if j[1] == "Noun":
            wine.review_token[i] = wine.review_token[i] + " " + j[0] 
        if j[1] == "Number":
            wine.review_token[i] = wine.review_token[i] + " " + j[0]     
        if j[1] == "Suffix":
            wine.review_token[i] = wine.review_token[i] + " " + j[0]
        if j[1] == "Verb":
            wine.review_token[i] = wine.review_token[i] + " " + j[0]

In [28]:
wine.head()

,name_ko,blog_text,review,review_token
0,더 롱 리틀 독 스파클링,더롱리틀독스파클링소유리20216291549복사이웃추가더롱리틀독스파클링단맛이없고깔끔한...,"스파클링.복숭아,살구,꽃,흰꽃,레몬.밝고맑은레몬컬러에레몬,살구,복숭아,흰꽃의아로마가...",스파클링 복숭아 살구 꽃 흰 꽃 레몬 밝고 맑은 레몬 컬러 에레몬 살구 복숭아 ...
1,루베롱 블랑,와인이야기하남포도내음와인아울렛와인샵와인소개화이트와인추천마레농클래식루베롱블랑2016포...,".헤이즐넛,꽃,아카시아,레몬.화사한노란색을띠며레몬,아카시아꽃,헤이즐넛향이어우러진다....",헤이즐넛 꽃 아카시아 레몬 화사한 노란색 띠 레몬 아카시아 꽃 헤이즐넛 향 어우...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천와인21닷컴이선정한7월이달의와인와인21202171950복사이웃추가조금늦게찾아...,"스파클링.꽃.반짝이는연한금빛의정교한거품을지녔으며,흰꽃패션프루츠,달콤한흰과일과은은한미...",스파클링 꽃 반짝이는 연한 금빛 정교한 거품 지녔으며 흰꽃패션프루츠 달콤한 흰 ...
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인마스카델타코제이픽와인산타20216212026복사이웃추가마스카델타코는이탈리...,".딸기,미네랄.밟은옅은분홍색에야생딸기젤리수박과같은달콤한향이풍부하고그뒤로향신료광물미네...",딸기 미네랄 밟은 옅은 분홍색 야생 딸기 젤리 수박 같은 달콤한 향 풍부하고 뒤...
4,"비에티, 바르베라 다스티 '트레 비그네'",2018비에띠바르베라다스티트레비녜2018데미안202142700복사이웃추가2018비에...,".체리,오크.바르베라다스티의포도는라크레나포도밭으로유명한아스티언덕주변인아그리아노테르메...",체리 오크 바르 베 라다 스티 포도 라크 레나 포도 밭 유명한 아스티 언덕 주변...


In [29]:
#wine["sorted_review_token"] = " "

In [ ]:
# for n in range(0,806):
    
#     vocab ={}
#     word_to_index ={}    
#     word_to_index_reverse = {}
#     sentence=[]
#     sentences=[]
#     i = 0
    
#     for word in word_tokenize(wine.review_token[n]):
#         if word not in vocab:
#             vocab[word] = 0
#         vocab[word] +=1
        
#         vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
    
#         continue;
    
#     for word,frequency in vocab_sorted:
#         i+=1
#         word_to_index[word]=i

#         continue;
    
#     for word in word_tokenize(wine.review_token[n]):
#         if word in word_to_index.keys():
#             if word_to_index[word] not in sentence:
#                  sentence.append(word_to_index[word])
#         sentence = sorted(sentence)
#         wine["sorted_review_token"][n] = sentence   
        
#         continue;
        
#     word_to_index_reverse = {v:k for k,v in word_to_index.items()}

#     for number in wine.sorted_review_token[n]:
#         sentences.append(word_to_index_reverse[number])
#         wine["review_token"][n] = sentences
#     continue;

###### review text 전체의 빈도수를 Dictionary로 나타냄

In [34]:
vocab ={}
for i in range(0,806):
    for word in word_tokenize(wine.review_token[i]):
        if word not in vocab:
            vocab[word] = 0
        vocab[word] +=1

In [35]:
print(vocab)

{'스파클링': 227, '복숭아': 144, '살구': 86, '꽃': 216, '흰': 56, '레몬': 125, '밝고': 14, '맑은': 13, '컬러': 87, '에레몬': 2, '아로마': 294, '느껴진다': 116, '입안': 203, '싱그러운': 24, '상큼': 59, '과일': 236, '기분': 103, '좋은': 334, '산미': 131, '포로': 6, '피어나는': 3, '와인': 2321, '헤이즐넛': 20, '아카시아': 29, '화사한': 25, '노란색': 18, '띠': 162, '향': 527, '어우러진다': 16, '부드럽고': 41, '섬세한': 60, '맛': 175, '생기있는': 7, '피니시': 34, '인상': 65, '적': 697, '반짝이는': 16, '연한': 12, '금빛': 24, '정교한': 5, '거품': 13, '지녔으며': 3, '흰꽃패션프루츠': 2, '달콤한': 96, '은은한': 65, '미네': 4, '럴': 4, '느낄수있다': 68, '우아하고': 34, '균형': 97, '감': 140, '뛰어난': 106, '샴페인': 251, '발': 12, '레드': 93, '마른과': 2, '세잔': 2, '생산': 573, '최상급': 17, '피노': 200, '뮈': 5, '누': 172, '샤르도네': 86, '품종': 278, '든': 19, '마리': 13, '스튜': 11, '아트': 7, '리지': 10, '널': 3, '블렌딩': 125, '비율': 12, '뀌': 3, '베드': 3, '헨느': 3, '여왕': 5, '뀌베': 9, '별칭': 5, '갖고있으며': 2, '도사': 9, '주는': 24, '8': 23, '7월': 14, '1867년': 5, '그': 71, '훗날': 1, '잉글랜드': 1, '스코틀랜드': 1, '의왕': 1, '되는': 90, '제임스': 12, '1': 45, '세': 12, '어머니': 3, '하며': 11, '많은': 3

In [36]:
print(vocab)

{'스파클링': 227, '복숭아': 144, '살구': 86, '꽃': 216, '흰': 56, '레몬': 125, '밝고': 14, '맑은': 13, '컬러': 87, '에레몬': 2, '아로마': 294, '느껴진다': 116, '입안': 203, '싱그러운': 24, '상큼': 59, '과일': 236, '기분': 103, '좋은': 334, '산미': 131, '포로': 6, '피어나는': 3, '와인': 2321, '헤이즐넛': 20, '아카시아': 29, '화사한': 25, '노란색': 18, '띠': 162, '향': 527, '어우러진다': 16, '부드럽고': 41, '섬세한': 60, '맛': 175, '생기있는': 7, '피니시': 34, '인상': 65, '적': 697, '반짝이는': 16, '연한': 12, '금빛': 24, '정교한': 5, '거품': 13, '지녔으며': 3, '흰꽃패션프루츠': 2, '달콤한': 96, '은은한': 65, '미네': 4, '럴': 4, '느낄수있다': 68, '우아하고': 34, '균형': 97, '감': 140, '뛰어난': 106, '샴페인': 251, '발': 12, '레드': 93, '마른과': 2, '세잔': 2, '생산': 573, '최상급': 17, '피노': 200, '뮈': 5, '누': 172, '샤르도네': 86, '품종': 278, '든': 19, '마리': 13, '스튜': 11, '아트': 7, '리지': 10, '널': 3, '블렌딩': 125, '비율': 12, '뀌': 3, '베드': 3, '헨느': 3, '여왕': 5, '뀌베': 9, '별칭': 5, '갖고있으며': 2, '도사': 9, '주는': 24, '8': 23, '7월': 14, '1867년': 5, '그': 71, '훗날': 1, '잉글랜드': 1, '스코틀랜드': 1, '의왕': 1, '되는': 90, '제임스': 12, '1': 45, '세': 12, '어머니': 3, '하며': 11, '많은': 3

###### 빈도수 순으로 나열함

In [37]:
vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

In [38]:
print(vocab_sorted)

[('와인', 2321), ('적', 697), ('하', 580), ('생산', 573), ('향', 527), ('포도', 387), ('좋은', 334), ('오크', 329), ('등', 319), ('지역', 308), ('아로마', 294), ('너리', 286), ('품종', 278), ('체리', 277), ('샴페인', 251), ('들', 243), ('과일', 236), ('스파클링', 227), ('꽃', 216), ('입안', 203), ('피노', 200), ('된', 198), ('매력', 192), ('타닌', 188), ('로', 182), ('숙성', 177), ('맛', 175), ('자두', 174), ('누', 172), ('잘', 164), ('부드러운', 163), ('띠', 162), ('100', 160), ('레이블', 158), ('프랑스', 155), ('요리', 149), ('미가', 149), ('느낌', 145), ('복숭아', 144), ('이름', 143), ('과실', 142), ('감', 140), ('바닐라', 140), ('이탈리아', 139), ('미네랄', 136), ('여운', 134), ('음식', 133), ('산미', 131), ('자리', 129), ('의향', 127), ('미를', 126), ('붉은', 126), ('개', 126), ('선물', 126), ('레몬', 125), ('블렌딩', 125), ('사용', 125), ('추천', 125), ('딸기', 124), ('같은', 124), ('바디', 122), ('익은', 121), ('이', 121), ('스타일', 121), ('복합', 121), ('다양한', 120), ('있는', 119), ('일향', 117), ('느껴진다', 116), ('배', 116), ('품질', 114), ('해', 113), ('블랙베리', 113), ('소비', 111), ('풍부한', 110), ('신선한', 109), ('블랙

###### 빈도수가 1이하인 단어 삭제

In [39]:
word_to_index = {}
i = 0
for word, frequency in vocab_sorted:
    if frequency > 1:
        i += 1
        word_to_index[word] = i

In [40]:
print(word_to_index)

{'와인': 1, '적': 2, '하': 3, '생산': 4, '향': 5, '포도': 6, '좋은': 7, '오크': 8, '등': 9, '지역': 10, '아로마': 11, '너리': 12, '품종': 13, '체리': 14, '샴페인': 15, '들': 16, '과일': 17, '스파클링': 18, '꽃': 19, '입안': 20, '피노': 21, '된': 22, '매력': 23, '타닌': 24, '로': 25, '숙성': 26, '맛': 27, '자두': 28, '누': 29, '잘': 30, '부드러운': 31, '띠': 32, '100': 33, '레이블': 34, '프랑스': 35, '요리': 36, '미가': 37, '느낌': 38, '복숭아': 39, '이름': 40, '과실': 41, '감': 42, '바닐라': 43, '이탈리아': 44, '미네랄': 45, '여운': 46, '음식': 47, '산미': 48, '자리': 49, '의향': 50, '미를': 51, '붉은': 52, '개': 53, '선물': 54, '레몬': 55, '블렌딩': 56, '사용': 57, '추천': 58, '딸기': 59, '같은': 60, '바디': 61, '익은': 62, '이': 63, '스타일': 64, '복합': 65, '다양한': 66, '있는': 67, '일향': 68, '느껴진다': 69, '배': 70, '품질': 71, '해': 72, '블랙베리': 73, '소비': 74, '풍부한': 75, '신선한': 76, '블랙': 77, '의': 78, '뛰어난': 79, '사과': 80, '질감': 81, '허브': 82, '의미': 83, '미와': 84, '하기': 85, '기분': 86, '빛': 87, '생산자': 88, '입니다': 89, '우아한': 90, '밸리': 91, '카베': 92, '균형': 93, '트러스': 94, '달콤한': 95, '오렌지': 96, '스페인': 97, '실향': 98, '레드': 99, '색': 1

###### 인코딩한 결과가 들어갈 새로운 열 만듦

In [41]:
wine["sorted_review_token"] = " "

###### 중복 제거, 빈도수에 다른 우선순위대로 나열 

In [42]:
for i in range(0,806):
    sentence=[]
    for word in word_tokenize(wine.review_token[i]):
         if word in word_to_index.keys():
                if word_to_index[word] not in sentence:
                    sentence.append(word_to_index[word])
    sentence = sorted(sentence)
    wine["sorted_review_token"][i] = sentence

In [43]:
wine.head()

,name_ko,blog_text,review,review_token,sorted_review_token
0,더 롱 리틀 독 스파클링,더롱리틀독스파클링소유리20216291549복사이웃추가더롱리틀독스파클링단맛이없고깔끔한...,"스파클링.복숭아,살구,꽃,흰꽃,레몬.밝고맑은레몬컬러에레몬,살구,복숭아,흰꽃의아로마가...",스파클링 복숭아 살구 꽃 흰 꽃 레몬 밝고 맑은 레몬 컬러 에레몬 살구 복숭아 ...,"[1, 7, 11, 17, 18, 19, 20, 39, 48, 55, 69, 86,..."
1,루베롱 블랑,와인이야기하남포도내음와인아울렛와인샵와인소개화이트와인추천마레농클래식루베롱블랑2016포...,".헤이즐넛,꽃,아카시아,레몬.화사한노란색을띠며레몬,아카시아꽃,헤이즐넛향이어우러진다....",헤이즐넛 꽃 아카시아 레몬 화사한 노란색 띠 레몬 아카시아 꽃 헤이즐넛 향 어우...,"[1, 2, 5, 19, 20, 27, 32, 55, 148, 167, 257, 3..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천와인21닷컴이선정한7월이달의와인와인21202171950복사이웃추가조금늦게찾아...,"스파클링.꽃.반짝이는연한금빛의정교한거품을지녔으며,흰꽃패션프루츠,달콤한흰과일과은은한미...",스파클링 꽃 반짝이는 연한 금빛 정교한 거품 지녔으며 흰꽃패션프루츠 달콤한 흰 ...,"[3, 4, 13, 15, 16, 17, 18, 19, 21, 25, 29, 42,..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인마스카델타코제이픽와인산타20216212026복사이웃추가마스카델타코는이탈리...,".딸기,미네랄.밟은옅은분홍색에야생딸기젤리수박과같은달콤한향이풍부하고그뒤로향신료광물미네...",딸기 미네랄 밟은 옅은 분홍색 야생 딸기 젤리 수박 같은 달콤한 향 풍부하고 뒤...,"[1, 3, 5, 20, 45, 46, 59, 60, 95, 98, 153, 221..."
4,"비에티, 바르베라 다스티 '트레 비그네'",2018비에띠바르베라다스티트레비녜2018데미안202142700복사이웃추가2018비에...,".체리,오크.바르베라다스티의포도는라크레나포도밭으로유명한아스티언덕주변인아그리아노테르메...",체리 오크 바르 베 라다 스티 포도 라크 레나 포도 밭 유명한 아스티 언덕 주변...,"[3, 4, 5, 6, 8, 10, 14, 20, 24, 26, 32, 46, 52..."


###### value 값으로 key값을 찾기 위해 dictionary를 뒤집음

In [44]:
word_to_index_reverse = {v:k for k,v in word_to_index.items()}

###### 인코딩이  따로 필요 없기 때문에 한국어로 다시 원상복귀함

In [45]:
for i in range(0,806):
    sentence=[]
    for word in wine.sorted_review_token[i]:
         sentence.append(word_to_index_reverse[word])
    wine["review_token"][i] = sentence

###### 필요 없어진 열 삭제

In [46]:
del wine["review"]
del wine["sorted_review_token"]

In [47]:
wine.head()

,name_ko,blog_text,review_token
0,더 롱 리틀 독 스파클링,더롱리틀독스파클링소유리20216291549복사이웃추가더롱리틀독스파클링단맛이없고깔끔한...,"[와인, 좋은, 아로마, 과일, 스파클링, 꽃, 입안, 복숭아, 산미, 레몬, 느껴..."
1,루베롱 블랑,와인이야기하남포도내음와인아울렛와인샵와인소개화이트와인추천마레농클래식루베롱블랑2016포...,"[와인, 적, 향, 꽃, 입안, 맛, 띠, 레몬, 인상, 섬세한, 부드럽고, 피니시..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천와인21닷컴이선정한7월이달의와인와인21202171950복사이웃추가조금늦게찾아...,"[하, 생산, 품종, 샴페인, 들, 과일, 스파클링, 꽃, 피노, 로, 누, 감, ..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인마스카델타코제이픽와인산타20216212026복사이웃추가마스카델타코는이탈리...,"[와인, 하, 향, 입안, 미네랄, 여운, 딸기, 같은, 달콤한, 실향, 향신료, ..."
4,"비에티, 바르베라 다스티 '트레 비그네'",2018비에띠바르베라다스티트레비녜2018데미안202142700복사이웃추가2018비에...,"[하, 생산, 향, 포도, 오크, 지역, 체리, 입안, 타닌, 숙성, 띠, 여운, ..."


###### 위와 마찬가지로 blog_text 열을  토큰화한 뒤 원하는 품사를 가진 단어만 남겨둘 새로운 열 추가

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
wine["blog_token"] = " "

###### 원하는 품사(의미를 가진 품사)만 새로운 열에 추가함

In [ ]:
pos_okt = okt.pos(wine.blog_text[1], norm=True, stem=True)

In [ ]:
for i in range(0,806):
    pos = okt.pos(wine.blog_text[i], norm=True, stem=True)
    for j in pos:
        if j[1] == "Verb":
            wine.blog_token[i] = wine.blog_token[i] + " " + j[0] 
        if j[1] == "Noun":
            wine.blog_token[i] = wine.blog_token[i] + " " + j[0] 
        if j[1] == "Adjective":
            wine.blog_token[i] = wine.blog_token[i] + " " + j[0] 

###### 단어의 빈도수를 확인하고, 블로그의 특성상 존재하는 관련없는 말, 의미없는 말을 제거하기 위해 빈도수가 3이 넘는 값만 중복을 제거하여 남겨둠

In [ ]:
wine.head()

## ------------------------------------------여기까지돌리고 복사본 만들기!

In [ ]:
blog_vocab ={}
for i in range(0,806):
    for word in word_tokenize(wine.blog_token[i]):
        if word not in blog_vocab:
            blog_vocab[word] = 0
        blog_vocab[word] +=1

In [ ]:
print(blog_vocab)

In [ ]:
blog_vocab_sorted = sorted(blog_vocab.items(), key=lambda x: x[1], reverse=True)

In [ ]:
print(blog_vocab_sorted)

In [ ]:
blog_word_to_index = {}
i = 0
for word, frequency in blog_vocab_sorted:
    if frequency > 3:
        i += 1
        word_to_index[word] = i

In [ ]:
print(blog_word_to_index)

In [ ]:
wine["sorted_blog_token"] = " "

In [ ]:
for i in range(0,806):
    sentence=[]
    for word in word_tokenize(wine.blog_token[i]):
         if word in blog_word_to_index.keys():
                if blog_word_to_index[word] not in sentence:
                    sentence.append(blog_word_to_index[word])
    sentence = sorted(sentence)
    wine["sorted_blog_token"][i] = sentence

In [ ]:
wine.head()

In [ ]:
blog_word_to_index_reverse = {v:k for k,v in blog_word_to_index.items()}

In [ ]:
for i in range(0,806):
    sentence=[]
    for word in wine.sorted_blog_token[i]:
         sentence.append(blog_word_to_index_reverse[word])
    wine["review_token"][i] = sentence

In [ ]:
del wine["blog_text"]
del wine["sorted_blog_token"]

In [ ]:
wine.head()

In [ ]:
#내일 둘 코드 보고 할 일!
#코드합치기
#token 두 열 합치기 (이때 중복 제거)
#review, blog text 열 제거
#csv형태로 만들기
#프론트 엔드 질문 생각

In [ ]:
wine["tokenizing"] = wine["review_token"] + wine["blog_token"]

#이때 중복제거해야됨..

In [ ]:
del wine.blog_text
del wine.review